In [1]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn import metrics

%matplotlib inline

In [2]:
# Read in test data
df = pd.read_csv('./datasets/test.csv')
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [3]:
df.isnull().sum().sort_values(ascending=False)

Pool QC         875
Misc Feature    838
Alley           821
Fence           707
Fireplace Qu    422
               ... 
Heating QC        0
Heating           0
Yr Sold           0
Bsmt Unf SF       0
Id                0
Length: 80, dtype: int64

<a id='feat'></a>
## Features Classification

In [4]:
# type_df: dictionary of column's types
type_df = pd.DataFrame(df.dtypes, columns= {'type':0})

# numeric_cols: list of column names with numeric type (int, float)
numeric_cols = list(type_df.loc[(type_df['type']== type(1))|(type_df['type']== type(1.1)), :].index)

# quantatative_cols: list of column names with quantatative data type
quantatative_cols = numeric_cols[3:5]+numeric_cols[9:26]+numeric_cols[27:36]+numeric_cols[38:]

# categorical_cols: list of numerical columns that are categorical
categorical_cols = [numeric_cols[2]] + numeric_cols[5:9] + [numeric_cols[26]] + numeric_cols[36:38]

# non-numeric cols:
non_numeric_cols = list(type_df.loc[(type_df['type']== type_df.loc[ 'MS Zoning' , :][0] ), :].index)

<a id='quan'></a>
## Quantatative Features

In [5]:
# Check if there are null values
df[quantatative_cols].isnull().sum().sort_values()

Full Bath            0
Screen Porch         0
3Ssn Porch           0
Enclosed Porch       0
Open Porch SF        0
Wood Deck SF         0
Garage Area          0
Garage Cars          0
Fireplaces           0
TotRms AbvGrd        0
Kitchen AbvGr        0
Bedroom AbvGr        0
Half Bath            0
Misc Val             0
Bsmt Half Bath       0
Bsmt Full Bath       0
Gr Liv Area          0
Low Qual Fin SF      0
2nd Flr SF           0
1st Flr SF           0
Total Bsmt SF        0
Bsmt Unf SF          0
BsmtFin SF 2         0
BsmtFin SF 1         0
Lot Area             0
Pool Area            0
Mas Vnr Area         1
Lot Frontage       160
dtype: int64

In [6]:
# Examine lot related features and 'Lot Frontage' == null

df.loc[df['Lot Frontage'].isnull(),
       ['Lot Frontage','Lot Area','Street', 'Alley' ,'Lot Shape' ,'Lot Config']].head()

,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Lot Config
1,NaN,9662,Pave,NaN,IR1,Inside
4,NaN,9500,Pave,NaN,IR1,Inside
7,NaN,9286,Pave,NaN,IR1,CulDSac
13,NaN,7976,Pave,NaN,Reg,Inside
20,NaN,8246,Pave,NaN,IR1,Inside


In [7]:
# Make dictionary of 'Lot Config' items and their corresponding 'Lot Frontage' average

Lot_Frontage_NA_fill=dict(zip(list(df.loc[:, ['Lot Frontage','Lot Config']].groupby('Lot Config')\
                                   .mean()['Lot Frontage'].keys()),
     list(df.loc[:, ['Lot Frontage','Lot Config']].groupby('Lot Config').mean()['Lot Frontage'].values)))

In [8]:
def lf_filler(row):
    
    """
    Replace null value in 'Lot Frontage column" with the average 'Lot Frontage' for the corresponding 'Lot Config'
    
    Parameter:
        - row: series/ ex) 'dataframe.apply(lf_filler, axis = 1)'
    """
    
    if np.isnan(row['Lot Frontage']):
        fill = Lot_Frontage_NA_fill[row['Lot Config']]
        
    else:
        fill = row['Lot Frontage']
    return fill

In [9]:
df['Lot Frontage'] = df.apply(lf_filler, axis = 1)

In [10]:
df.loc[:, ['Lot Frontage','Lot Config']].isnull().sum()

Lot Frontage    0
Lot Config      0
dtype: int64

In [11]:
def mean_filler(cols, inplace):
    """
    Replace na values with its column's mean
    
    Paramters:
        cols: list/ list of column names
        inplace: boolean/ inplace = True or False
    """    
    for col in cols:
        df[col].fillna(df[col].mean(), inplace = inplace)

In [12]:
# Replace the NA values from the rest of the quatatative columns with its mean 

mean_filler(quantatative_cols, True)

In [13]:
# Check if there is any NA values
df.loc[:,quantatative_cols].isnull().sum().sort_values(ascending = False)

Misc Val           0
Pool Area          0
Lot Area           0
Mas Vnr Area       0
BsmtFin SF 1       0
BsmtFin SF 2       0
Bsmt Unf SF        0
Total Bsmt SF      0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bath     0
Bsmt Half Bath     0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
TotRms AbvGrd      0
Fireplaces         0
Garage Cars        0
Garage Area        0
Wood Deck SF       0
Open Porch SF      0
Enclosed Porch     0
3Ssn Porch         0
Screen Porch       0
Lot Frontage       0
dtype: int64

<a id='cat'></a>
## Categorical Features

In [14]:
df[categorical_cols].isnull().sum()

MS SubClass        0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Garage Yr Blt     45
Mo Sold            0
Yr Sold            0
dtype: int64

In [15]:
df[non_numeric_cols].isnull().sum().sort_values(ascending = False)

Pool QC           875
Misc Feature      838
Alley             821
Fence             707
Fireplace Qu      422
Garage Cond        45
Garage Qual        45
Garage Finish      45
Garage Type        44
BsmtFin Type 2     25
Bsmt Cond          25
Bsmt Exposure      25
BsmtFin Type 1     25
Bsmt Qual          25
Mas Vnr Type        1
Electrical          1
Neighborhood        0
Land Slope          0
Condition 1         0
Condition 2         0
Sale Type           0
Utilities           0
Lot Config          0
House Style         0
Land Contour        0
Lot Shape           0
Street              0
Bldg Type           0
Foundation          0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Exter Qual          0
Exter Cond          0
Heating             0
Heating QC          0
Central Air         0
Kitchen Qual        0
Functional          0
Paved Drive         0
MS Zoning           0
dtype: int64

#### Garage Yr Blt 

In [16]:
def Garage_Yr_Blt_filler(row):
    
    """
    Replace each row of 'Garage Yr Blt' column as following:
        NA: 'no_garage'
        >= 1975: 'new_garage'
        < 1975: 'old_garage'
    
    Parameter:
        - row: series/ ex) 'dataframe.apply(lf_filler, axis = 1)'
    """
    
    if np.isnan(row['Garage Yr Blt']):
        fill = 'no_garage'
        
    elif row['Garage Yr Blt'] >= 1975:
        fill = 'new_garage'
        
    else:
        fill = 'old_garage'
    return fill

In [17]:
df['Garage Yr Blt'] = df.apply(Garage_Yr_Blt_filler, axis = 1)

In [18]:
df['Garage Yr Blt'].value_counts()

new_garage    458
old_garage    376
no_garage      45
Name: Garage Yr Blt, dtype: int64

In [19]:
df = pd.get_dummies(df, columns= ['Garage Yr Blt'], drop_first=True)

#### Yr Sold - Year Built

In [20]:
# Create 'age_when_sold' column by df[Yr Sold] - df[Year Built]

df['age_when_sold'] = df['Yr Sold'] - df['Year Built']

In [21]:
# Drop 'Yr Sold' and 'Year Built' as the two columns are correlated to 'age_when_sold'

df = df.drop(['Yr Sold', 'Year Built'], axis = 1)

#### Neighborhood
- Average Sale Price in the train data set = 187579.71007846566
- This value was used as a threshold to divide the neighborhoods into two groups (g_nbr, w_nbr)

In [22]:
g_nbr = ['Blmngtn', 'ClearCr', 'CollgCr', 'Crawfor', 'Gilbert', 'Greens',
           'GrnHill', 'NWAmes', 'NoRidge', 'NridgHt', 'SawyerW', 'Somerst',
           'StoneBr', 'Timber', 'Veenker']

In [23]:
b_nbr = ['Blueste', 'BrDale', 'BrkSide', 'Edwards', 'IDOTRR', 'Landmrk',
       'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'OldTown', 'SWISU',
       'Sawyer'] 

In [24]:
"""
Binarize 'Neighborhood' column as following:
    - 1: the average sale price of the property in the neighborhood is higer than overall average.
    - 0: the average sale price of the property in the neighborhood is higer than overall average.
"""

df['Neighborhood'] = df['Neighborhood'].apply(lambda x: 1 if x in g_nbr else 0)

In [25]:
df['Neighborhood'].value_counts()

1    441
0    438
Name: Neighborhood, dtype: int64

#### Central Air, Street
    - These two features have two items each.
    - Binarize them into 0 and 1

In [26]:
## Central Air
df['Central Air'] = df['Central Air'].apply(lambda x: 1 if x == 'Y' else 0)

In [27]:
## Street
df['Street'] = df['Street'].apply(lambda x: 1 if x == 'Pave' else 0)

#### Columns with ranking items (ex. good, average, bad)
    - Columns: 'Heating QC', 'Kitchen Qual', 'Exter Qual', 'Exter Cond'
    - Assign numerical score

In [28]:
def unique_vals(df):
    """
    Return dictionary that has
         key == column names
         value == the column's unique values
    """
    
    val_dic = {}
    for col in df.columns:
        val_dic.update({col: df[col].unique()})
    return val_dic

In [29]:
# Make dictionary to turn string score to numerical score

str_score_5 = ['Ex','Gd','TA','Fa', 'Po']
num_score_5 = np.arange(5,-1,-1)
score_5_dict = dict(zip(str_score_5, num_score_5))

str_score_4 = ['Ex','Gd','TA','Fa']
num_score_4 = np.arange(4,-1,-1)
score_4_dict = dict(zip(str_score_4, num_score_4))

In [30]:
# View unique values
ranking_cols =['Heating QC', 'Kitchen Qual',
                      'Exter Qual', 'Exter Cond']

score_df = df.loc[:, ranking_cols]
unique_vals(score_df)

{'Exter Cond': array(['Fa', 'TA', 'Gd', 'Ex', 'Po'], dtype=object),
 'Exter Qual': array(['TA', 'Gd', 'Fa', 'Ex'], dtype=object),
 'Heating QC': array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object),
 'Kitchen Qual': array(['Fa', 'TA', 'Gd', 'Ex', 'Po'], dtype=object)}

In [31]:
# Turn string score to numerical score

df['Exter Cond'] = df['Exter Cond'].map(score_5_dict)
df['Heating QC'] = df['Heating QC'].map(score_5_dict)
df['Kitchen Qual'] = df['Kitchen Qual'].map(score_4_dict)
df['Exter Qual'] = df['Exter Qual'].map(score_4_dict)

<a id='cat_clean_NA'></a>

### Cleaning NA values

In [32]:
df.isnull().sum().sort_values(ascending=False).head(18)

Pool QC           875
Misc Feature      838
Alley             821
Fence             707
Fireplace Qu      422
Garage Cond        45
Garage Finish      45
Garage Qual        45
Garage Type        44
Bsmt Exposure      25
Bsmt Cond          25
Bsmt Qual          25
BsmtFin Type 1     25
BsmtFin Type 2     25
Electrical          1
Mas Vnr Type        1
Kitchen Qual        1
Utilities           0
dtype: int64

Observation:

- Meaning of all NA values in the columns are specified in the data dictionary.(http://jse.amstat.org/v19n3/decock/DataDocumentation.txt)
- NA values in 'Misc Feature' means no miscellaneous feature not covered in other categories. This does not seem significant. Therefore, drop this column.

Plan:

- Drop 'Misc Feature' column.
- Turn NA value in rest of the columns to a string that represents its meaning.
- The columns are to be dummied out.

#### Misc Feature

In [33]:
df = df.drop('Misc Feature', axis = 1)

#### Columns that NA values will be given a string value. (To be dummied)

In [34]:
null_cols = ['Pool QC', 'Alley', 'Fence',
             'Fireplace Qu', 'Garage Finish',
             'Garage Qual', 'Garage Cond', 'Garage Type',
             'Bsmt Exposure', 'BsmtFin Type 2', 'Bsmt Qual',
             'Bsmt Cond', 'BsmtFin Type 1', 'Mas Vnr Type']

In [35]:
df[null_cols].isnull().sum()

Pool QC           875
Alley             821
Fence             707
Fireplace Qu      422
Garage Finish      45
Garage Qual        45
Garage Cond        45
Garage Type        44
Bsmt Exposure      25
BsmtFin Type 2     25
Bsmt Qual          25
Bsmt Cond          25
BsmtFin Type 1     25
Mas Vnr Type        1
dtype: int64

In [36]:
for null_col in null_cols:
    df[null_col] = df.loc[:,null_col].fillna('none')

In [37]:
df[null_cols].isnull().sum()

Pool QC           0
Alley             0
Fence             0
Fireplace Qu      0
Garage Finish     0
Garage Qual       0
Garage Cond       0
Garage Type       0
Bsmt Exposure     0
BsmtFin Type 2    0
Bsmt Qual         0
Bsmt Cond         0
BsmtFin Type 1    0
Mas Vnr Type      0
dtype: int64

In [38]:
df.isnull().sum().sort_values(ascending=False).head(18)

Electrical        1
Kitchen Qual      1
age_when_sold     0
Exter Qual        0
Roof Style        0
Roof Matl         0
Exterior 1st      0
Exterior 2nd      0
Mas Vnr Type      0
Mas Vnr Area      0
Exter Cond        0
Bsmt Unf SF       0
Foundation        0
Bsmt Qual         0
Bsmt Cond         0
Bsmt Exposure     0
BsmtFin Type 1    0
BsmtFin SF 1      0
dtype: int64

#### Electrical, Kitchen Qual columns did not have NA in the train set.
    - Kitchen Qual's NA -> 1
    - Electrical's NA -> 'Mix'

In [39]:
df.loc[:,'Electrical']

0      FuseP
1      SBrkr
2      SBrkr
3      SBrkr
4      SBrkr
       ...  
874    SBrkr
875    SBrkr
876    SBrkr
877    SBrkr
878    SBrkr
Name: Electrical, Length: 879, dtype: object

In [40]:
df.loc[:,'Kitchen Qual']

0      1.0
1      2.0
2      3.0
3      2.0
4      2.0
      ... 
874    2.0
875    3.0
876    2.0
877    2.0
878    2.0
Name: Kitchen Qual, Length: 879, dtype: float64

In [41]:
df['Kitchen Qual'] = df.loc[:,'Kitchen Qual'].fillna(1)

In [42]:
df['Kitchen Qual'].unique()

array([1., 2., 3., 4.])

In [43]:
df['Electrical'].unique()

array(['FuseP', 'SBrkr', 'FuseA', 'FuseF', nan], dtype=object)

In [44]:
df['Electrical'] = df.loc[:,'Electrical'].fillna('FuseP')

In [45]:
df['Electrical'].unique()

array(['FuseP', 'SBrkr', 'FuseA', 'FuseF'], dtype=object)

<a id='cat_dummy'></a>

### Get Dummy Variables (Categorical Features Only)

In [46]:
to_be_dummy_cols = df.loc[:, df.dtypes == object].columns

In [47]:
to_be_dummy_cols

Index(['MS Zoning', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Bsmt Qual', 'Bsmt Cond',
       'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating',
       'Electrical', 'Functional', 'Fireplace Qu', 'Garage Type',
       'Garage Finish', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Pool QC',
       'Fence', 'Sale Type'],
      dtype='object')

In [48]:
df = pd.get_dummies(df, columns= to_be_dummy_cols, drop_first=True)

<a id='feat_eng'></a>

### Feature Engineering (Get Interaction Terms)

- Create interaction terms with qunatatative features (numerical quantative columns, columns with numerical ranking score).

In [49]:
# Generate list of column names to create interaction terms with
interact_term_cols = numeric_cols + ranking_cols + ['age_when_sold']

In [50]:
# Remove columns that are not needed
cols_to_remove = ['Id', 'PID', 'Garage Yr Blt', 'Yr Sold', 'Year Built']
for col in cols_to_remove:
    interact_term_cols.remove(col)

In [51]:
# Make dataframe of features to make interaction terms with
X_interact_df = df[interact_term_cols]

In [53]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_interact = poly.fit_transform(X_interact_df)
poly_df = pd.DataFrame(X_interact, columns = poly.get_feature_names(interact_term_cols))

/Users/dae/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:1263: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for c in combinations)


In [54]:
# merge interaction terms to the original dataframe

df = pd.merge(df, poly_df, on=df.index).drop('key_0', axis =1)

In [55]:
fin_cols = ['Neighborhood', 'MS Zoning_RM', 'Land Contour_HLS',
       'Roof Style_Mansard', 'Exterior 1st_BrkFace', 'Bsmt Exposure_Gd',
       'Functional_Typ', 'Sale Type_New', 'MS SubClass Kitchen AbvGr',
       'Lot Frontage Overall Qual', 'Lot Area Overall Qual',
       'Lot Area Screen Porch', 'Lot Area Heating QC', 'Overall Qual^2',
       'Overall Qual Gr Liv Area', 'Overall Qual Bsmt Full Bath',
       'Overall Qual TotRms AbvGrd', 'Overall Qual Fireplaces',
       'Overall Qual Garage Area', 'Overall Qual Exter Qual',
       'Overall Qual age_when_sold', 'Overall Cond Total Bsmt SF',
       'Overall Cond 1st Flr SF', 'Overall Cond Gr Liv Area',
       'Year Remod/Add^2', 'Mas Vnr Area Garage Cars',
       'Mas Vnr Area Pool Area', 'Mas Vnr Area Misc Val',
       'BsmtFin SF 1 Garage Cars', 'BsmtFin SF 1 Mo Sold',
       'BsmtFin SF 1 Heating QC', 'BsmtFin SF 2 Pool Area',
       'Total Bsmt SF Garage Cars', '1st Flr SF Heating QC',
       'Gr Liv Area Heating QC', 'Gr Liv Area Kitchen Qual',
       'Bsmt Full Bath Exter Qual', 'Bsmt Half Bath Misc Val',
       'Full Bath Wood Deck SF', 'Kitchen AbvGr age_when_sold',
       'Fireplaces Garage Cars', 'Fireplaces Exter Qual',
       'Garage Area Kitchen Qual', 'Kitchen Qual age_when_sold',
       'Exter Qual age_when_sold']

In [56]:
df_n = df[['Id']+fin_cols]

In [58]:
df_n.to_csv('./datasets/clean_kaggle.csv', index = False)